In [2]:
import tensorflow as tf
import numpy as np
import os
import splitfolders

splitfolders.ratio('tometo_disease\\training\PlantVillage', output='train_test', ratio=(0.8, 0.2))

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [6]:
# Building CNN layers (3 convolutional layers with max pooling, 1 dense layer, and a softmax output layer)
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, kernel_size = (3,3), activation='relu', input_shape=(255, 255, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax'),
])

# Compiling the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [9]:
# Using generator to input the training and validation images. Also, used image augmentation on the training data.
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1/255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
).flow_from_directory('train_test/train',
                      target_size=(255, 255),
                      batch_size=256,
                      class_mode='categorical')

val_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1/255,
).flow_from_directory('train_test/val',
                      target_size=(255, 255),
                      batch_size=64,
                      class_mode='categorical')

Found 12804 images belonging to 10 classes.
Found 3207 images belonging to 10 classes.


In [10]:
# Using checkpoints to save models
# checkpoint = tf.keras.callbacks.ModelCheckpoint('model-{epoch:03d}.model', monitor='val_loss', verbose=0,
#                                                 save_best_only=True, mode='auto')

# Fitting the model on the data. Saved the model in a variable to compare results
history = model.fit(train_generator, epochs=20, validation_data=val_generator)

Epoch 1/20
51/51 [==============================] - 312s 6s/step - loss: 1.9999 - accuracy: 0.2665 - val_loss: 1.8216 - val_accuracy: 0.3265
Epoch 2/20
51/51 [==============================] - 292s 6s/step - loss: 1.8151 - accuracy: 0.3484 - val_loss: 1.7722 - val_accuracy: 0.3807
Epoch 3/20
51/51 [==============================] - 310s 6s/step - loss: 1.6268 - accuracy: 0.4273 - val_loss: 1.6453 - val_accuracy: 0.4203
Epoch 4/20
51/51 [==============================] - 316s 6s/step - loss: 1.5994 - accuracy: 0.4531 - val_loss: 1.4452 - val_accuracy: 0.4861
Epoch 5/20
51/51 [==============================] - 312s 6s/step - loss: 1.3902 - accuracy: 0.5174 - val_loss: 1.3473 - val_accuracy: 0.5610
Epoch 6/20
51/51 [==============================] - 311s 6s/step - loss: 1.2110 - accuracy: 0.5818 - val_loss: 1.2168 - val_accuracy: 0.5688
Epoch 7/20
51/51 [==============================] - 308s 6s/step - loss: 0.9653 - accuracy: 0.6654 - val_loss: 0.9317 - val_accuracy: 0.6810
Epoch 8/20
51

In [11]:
model.save('model.h5')